#### 1. Import all libraries, modules and packages.

In [1]:
#preprocess and mine the text data
import pandas as pd
import re
import string
import docx2txt
from pdfminer.high_level import extract_text

In [2]:
#compute similarity between resume and job role
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bhramar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### 2. Define functions to extract text from pdf and doc.

In [3]:
#from pdf
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

#from docx
def extract_text_from_docx(docx_path):
    txt = docx2txt.process(docx_path)
    if txt: 
        return txt.replace('\t',' ')
    return None

#### 3. Define functions to extract candidate information.

In [4]:
#extraction of name
#match  = re.search(r'(\b[A-Z][a-z]+\b)\s(\b[A-Z][a-z]+\b)', s)

def extract_name_from_resume(s):
    name = None
    pattern = r"(\b[A-Z][a-z]+\b)\s(\b[A-Z][a-z]+\b)"
    match = re.search(pattern,s)
    if match:
        name = match.group()
    return name

In [5]:
#Extraction of Phone number
#match = re.findall('''(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)[-\.\s]*\d{3}[-\.\s]??\d{4}|\d{5}[-\.\s]??\d{4})''',s)

def extract_contact_number_from_resume(s):
    contact_number = None
    pattern = r"(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)[-\.\s]*\d{3}[-\.\s]??\d{4}|\d{5}[-\.\s]??\d{4})"
    match = re.search(pattern, s)
    if match:
        contact_number = match.group()
    return contact_number

In [6]:
#Extraction of email
#match = re.findall(r'[\w\.-]+@[\w\.-]+',s)   

def extract_email_from_resume(s):
    email = None
    pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    match = re.search(pattern,s)
    if match:
        email = match.group()
    return email

In [7]:
#Extraction of Education
#match  = re.search(r"(?i)(?:Bsc|\bB\.\[A-Za-z]+|\bM\.\[A-Za-z]+|\bPh\.D\.\[A-Za-z]+|\bBachelor(?:'s)?|\bMaster(?:'s)?|\bPh\.D)\s(?:\w+\s)*\w+|B\.[A-Za-z]+", s)

def extract_education_from_resume(s):
    education = []
    pattern = r"(?i)(?:Bsc|\bM\.\[A-Za-z]+|\bPh\.D\.\[A-Za-z]+|\bBachelor(?:'s)?|\bBachelors(?:'s)?|\bMaster(?:'s)?|\bMasters(?:'s)?|\bPh\.D)\s(?:\w+\s)*\w+|B\.[A-Za-z]+"
    matches = re.findall(pattern, s)
    for match in matches:
        education.append(match.strip())
    return education

In [8]:
#Extraction of GPA
#match = re.findall([0-9]+\.\d?(\d))

def extract_gpa_from_resume(s):
    gpa = None
    pattern = r'[0-9]+\.\d?(\d)'
    match = re.search(pattern,s)
    if match:
        gpa = match.group()
    return gpa

In [9]:
#Extraction of Skills
def extract_skills_from_resume(s, skills_list):
    skills = []
    for skill in skills_list:
        pattern = r"\b{}\b".format(re.escape(skill))
        match = re.search(pattern, s, re.IGNORECASE)
        if match:
            skills.append(skill)
    return skills

#### 4. Load job descriptions into python dictionaries.

In [10]:
#key: job role, values: skill requirements

jobRoles = {
    'Frontend Engineer' : ['frontend','HTML', 'HTML5', 'CSS', 'CSS3', 'React','ReactJS','Angular','AngularJS','jQuery','Frameworks','Version Control','Responsive Design','UI','UX','UI/UX','REST','SEO','Communication','Teamwork','Creativity','Problem Solving','Quick Learner','Attention to Detail','Collaboration','Problem Solving','Writing Skills','Prototype','Troubleshoot','Optimization', 'Graphic Design', 'Testing', 'Debugging','Deployment', 'Web Development','Content Management','Fast Paced','Analytical', 'Multitasking'],
    'Backend Engineer' : ['backend','Java','Python','Ruby','Javascript','HTML','HTML5','CSS','CSS3','PHP','Database Management','APIs','Data Structures and Algorithms','CMS','Communication','Teamwork','Collaboration','Problem Solving','Time Management','Critical Thinking','Attention to Detail','Management','Troubleshoot','Debugging','Deployment','Web Development','UI','UX','UI/UX','Collboration','Reusable Code','Analytical','Time Management','Teamwork','Problem Solving'],
    'Fullstack Developer' : ['fullstack','HTML','HTML5','CSS','CSS3','JavaScript','XML','jQuery','Java','Python','PHP','C#','React','ReactJS','Angular','AngularJS','Git/Github','APIs','MySQL','MongoDB','Apache','Frameworks','Version Control','Responsive Design','UI/UX','UI','UX','REST','SEO','Communication','Teamwork','Collaboration','Problem Solving','Time Management','Critical Thinking','Attential to Detail','Management','Creativity','Problem Solving','Client Side Architecture','Server Side Architechture','Responsiveness','Responsive Design','Efficiency','Troubleshoot','Debug','Organizational','Analytical'],
    'Software Engineer' : ['software','Java','Ruby on Rails','C++','Python','Database Management','Data Structures and Algorithms','Object Oriented Programming','Operating Systems','Frameworks','ORM','SQL','JavaScript','Communication','Critical Thinking','Adaptability','Problem Solving','Teamwork','Collaboration','Time Management','SDLC','Problem Solving','Testing','Feasibility','Integration','Quality Assurance','Troubleshoot','Debug','Deployment','Documentation'],
    'Cybersecurity Engineer' : ['cybersecurity','Network Security','Software Testing','Operating Systems','Digital Forensics','Security Auditing','Ethical Hacking','Cloud Security','Communication','Collaboration','Critical Thinking','Adaptability','Problem Solving','Attention to Detail','Security Control','Security Requirements','Performance Reports','Network Data Analysis','Vulnerability Scanning Solutions','Antivirus Software','Security Software'],
    'Blockchain Engineer' : ['blockchain','C++', 'Golang', 'Java', 'Cryptography', 'Network Security', 'Data Structures', 'Blockchain Architecture', 'Web Development', 'Communication', 'Collaboration', 'Critical Thinking', 'Adaptability', 'Problem Solving', 'Attention to Detail','Blockchain','Client and Server Side Applications','Software Development','Open Source Projects', 'JavaScript','Data Structures','Cryptography','P2P','Bitcoin','Concurrency','Writing efficient','Safe Multithreaded Code'],
    'DevOps Engineer' : ['devops','Python', 'JavaScript', 'Ruby', 'Automation', 'Security', 'Software Testing', 'Linux', 'Scripting', 'System Integration', 'Cloud Skills', 'Risk Assessment', 'CI/CD','Communication', 'Management', 'Collaboration', 'Decision-Making', 'Problem Solving', 'Adaptability','Integrations','Level 2 Technical Support','Backend','Root Cause Analysis','System Troubleshooting','Maintenance','DevOps Engineer','SQL','Teamwork'],
    'Android Developer' : ['Android Studio', 'Android SDK', 'Android Testing', 'APIs', 'Java', 'Kotlin', 'XML', 'SQL', 'Firebase', 'JSON', 'Design', 'Git/GitHub', 'UI/UX', 'Version Control', 'Communication', 'Problem Solving', 'Collaboration', 'Time Management','Unit-Test code ','Bug Fixing ','Android','Android SDK','REST', 'JSON','Mobile Development'],
    'Database Administrator' : ['dba','SQL', 'LINUX', 'Oracle', 'UNIX', 'Microsoft Access', 'Windows OS', 'HTML','Communication', 'Problem Solving', 'Organization', 'Analytics', 'Business – focus','Design Database ','Implement Database','Data Security', 'Privacy', 'Integrity','Database Administrator','Database Standards ','Data Backup', 'Data Recovery', 'Data Security','Database Design','Documentation','Coding','DBA tools'],
    'Data Analyst' : ['SQL', 'R', 'Python', 'Tableau', 'Microsoft Excel', 'Mathematical Skills', 'Data Cleaning','Presentation', 'Critical Thinking', 'Communication', 'Problem Solving','Statistical Techniques','Data Collection Systems', 'Analytics','Data Analyst','Business Data Analyst','Data Models', 'Database Design Development', 'Data Mining','Segmentation Techniques','XML', 'JavaScript', 'ETL frameworks','Excel', 'SPSS', 'SAS','Analytical skills','Queries','Report Writing','Presenting Skills'],
    'Data Scientist' : ['Python', 'R', 'SQL', 'Mongo DB', 'MySQL', 'Regression', 'Vector Models', 'Tableau', 'Power BI', 'DS.js', 'BeautifulSoup', 'Pandas', 'Spacy', 'Machine Learning', 'Artificial Intelligence', 'NLP', 'Hadoop', 'Spark','Problem solving', 'Effective Communication', 'Intellectual Curiosity', 'Business Sense','Predictive Models ','Machine Learning Algorithms','Data Visualization','Data Scientist', 'Data Analyst','Data Mining','Scala', 'Java', 'C++','Communication','Presentation Skills'],
    'Data Engineer' : ['Python', 'Java', 'Scala', 'PostgreSQL', 'NoSQL', 'Hadoop', 'Spark', 'Apache Kafka', 'Azure', 'Google Cloud', 'Git', 'Amazon Redshift', 'BigQuery','Communication', 'Collaboration', 'Critical Thinking','Prescriptive and Predictive modeling','Algorithms','Prototypes','Data quality and reliability','Data engineer','data models', 'data mining', 'segmentation techniques','Great numerical and analytical skills'],
    'Business Analyst' : ['SQL','R','Python','SPSS','SAS','Sage','Mathematics','Excel','Power BI','Tableau','Communication','Presentations','Problem Solving','Critical Thinking','Negotiation','Adaptability','Business Intellect','Reporting and Alerting','System Analysis','Maintain System','Quality Assurace','Visualization','Key Performance Indicators','Reporting','Writing Skills','Technical Writing'],
    'Machine Learning Engineer' : ['Statistcal Analysis','Probability','Machine Learning','Model Evaluation','DevOps','CI/CD','AWS','Azure','Google Cloud Platform','Python','C++','Java','Communication','Problem Solving','Adaptability','Data Science Prototypes','Data Representation','Train Systems','Scikit Learn','Tensorflow','Keras','PyTorch','Data Structures','Data Modelling','Software Architecture','MathS','Probability','Python','R','Communication','Teamwork','Analytical Skills','Problem Solving'],
    'Artificial Intelligence Engineer' : ['Python', 'Java', 'R', 'C++','JavaScript','SQL','NoSQL','Apache Spark','Hadoop','SparkSQL','Apache Flink', 'Google Cloud Platform','AWS Azure','ML Models','AI Models','Communication','Collboration','Adaptability','Critical Thinking','Problem Solving','Domain Knowledge','Staistical Analysis','Algorithmic Models','Linear Algebra','Probability','Efficient Code','Big Data'],
}

#### 5. Define functions to match resume text to job description keywords.

In [11]:
#remove punctuation and stopwords

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if token not in string.punctuation]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

In [12]:
def calculate_cosine_similarity(text, keyword_list):
    
    #convert text and keyword list to strings
    text_str = ' '.join(text)
    keyword_str = ' '.join(keyword_list)
    
    #create TF-IDF vectorizer
    vectorizer = TfidfVectorizer()  
    #fit and transform the text and keyword list
    tfidf_matrix = vectorizer.fit_transform([text_str, keyword_str]) 
    
    #calculate cosine similarity
    similarity = cosine_similarity(tfidf_matrix)[0, 1]
    return similarity

In [13]:
def calculate_keyword_match(text, keywords):
    text_tokens = preprocess_text(text)
    keyword_match = {}
    for key, values in keywords.items():
        #calculate cosine similarity for each keyword list
        similarity_scores = [calculate_cosine_similarity(text_tokens, preprocess_text(keyword)) for keyword in values]
        
        # average similarity score for the keyword list
        avg_similarity = sum(similarity_scores) / len(similarity_scores)
        keyword_match[key] = avg_similarity
        
    #sort the dictionary based on average similarity score and return top 3 keys
    top_keys = sorted(keyword_match, key=keyword_match.get, reverse=True)[:3]
    return top_keys

#### 6. Load a resume and extract information.

In [14]:
res1=r"Resume_Kirti_Patel - Kirti Patel.pdf"
s1 = extract_text_from_pdf(res1)
print(s1)

Kirti Patel

A strategic, creative and innovative achiever with 2+ years of experience
in web development and creating engaging social media content that
resonates with targeted audiences.

kirtiipatel4@gmail.com
8655204810
Mumbai Maharashtra
https://www.linkedin.com/in/kirti-patel-
843b13220/

EXPERIENCE

EDUCATION

Content Manager (Freelance) , Swacardz

 Usha Mittal Institute of Technology -

May 2023 - Present , Indore

●

●

●

Collaborated with the client to develop content strategies tailored to

their speciﬁc goals and target audiences.

Created and managed high-quality content for social media

Conducted thorough research on industry trends and ensured timely

delivery of deliverables

● Managed content calendar and utilized analytics tools to track

content performance and make data-driven decisions for

improvement

Business Analyst Intern , TTBTE

September 2022 - March 2023 , Mumbai

Conducted competitive analysis and market research to identify

trends and opportunities f

In [15]:
#basic information

print("Name:", extract_name_from_resume(s1))
print("Contact Number:", extract_contact_number_from_resume(s1))
print("Email:", extract_email_from_resume(s1))
print("Education:", extract_education_from_resume(s1))
print("CGPA:", extract_gpa_from_resume(s1))

#skills for a specific role
skills_list =  ['frontend','HTML', 'HTML5', 'CSS', 'CSS3', 'React','ReactJS','Angular','AngularJS','jQuery','Frameworks','Version Control','Responsive Design','UI','UX','UI/UX','REST','SEO','Communication','Teamwork','Creativity','Problem Solving','Quick Learner','Attention to Detail','Collaboration','Problem Solving','Writing Skills','Prototype','Troubleshoot','Optimization', 'Graphic Design', 'Testing', 'Debugging','Deployment', 'Web Development','Content Management','Fast Paced','Analytical', 'Multitasking']
print("Skills that match specified role:", extract_skills_from_resume(s1, skills_list))

#roles that match candidate skills
print("Top roles that match candidate skills:", calculate_keyword_match(s1, jobRoles))

Name: Kirti Patel
Contact Number: 8655204810
Email: kirtiipatel4@gmail.com
Education: ['Bachelor of Technology']
CGPA: 9.43
Skills that match specified role: ['React', 'Optimization', 'Testing', 'Debugging', 'Web Development']
Top roles that match candidate skills: ['Frontend Engineer', 'Data Analyst', 'Blockchain Engineer']


#### 7. Connect to SQLite and create a database.

In [16]:
import csv, sqlite3

con = sqlite3.connect("SQLiteMagic.db")
cur = con.cursor()

In [17]:
%load_ext sql

In [18]:
%sql sqlite:///SQLiteMagic.db

In [20]:
%%sql

CREATE TABLE CANDIDATES (
Name VARCHAR(50),
Contact_Number VARCHAR(15),
Email TEXT,
Education TEXT,
CGPA REAL,
Role1 TEXT,
Role2 TEXT,
Role3 TEXT
);

 * sqlite:///SQLiteMagic.db
Done.


[]

In [21]:
%%sql

INSERT INTO CANDIDATES(Name, Contact_Number, Email, Education, CGPA, Role1, Role2, Role3) 
VALUES ('Kirti Patel', 8655204810, 'kirtiipatel4@gmail.com','Bachelor of Technology', 9.43, 'Frontend Engineer','Data Analyst', 'Blockchain Engineer');

 * sqlite:///SQLiteMagic.db
1 rows affected.


[]

#### 8. Extract and store information from more resumes.

In [22]:
res2=r"Vansh Verified Resume - Vansh Mehra.pdf"
s2 = extract_text_from_pdf(res2)
print("Name:", extract_name_from_resume(s2))
print("Contact Number:", extract_contact_number_from_resume(s2))
print("Email:", extract_email_from_resume(s2))
print("Education:", extract_education_from_resume(s2))
print("CGPA:", extract_gpa_from_resume(s2))
print("Top roles that match candidate skills:", calculate_keyword_match(s2, jobRoles))

Name: Vansh Mehra
Contact Number: 9711982402
Email: vansh_m@ch.iitr.ac.in
Education: ['B.Tech', 'B.Tech']
CGPA: 8.17
Top roles that match candidate skills: ['Data Analyst', 'Data Engineer', 'Data Scientist']


In [23]:
res3=r"ZeniaNouphalResume - Zenia Nouphal.pdf"
s3 = extract_text_from_pdf(res3)
print("Name:", extract_name_from_resume(s3))
print("Contact Number:", extract_contact_number_from_resume(s3))
print("Email:", extract_email_from_resume(s3))
print("Education:", extract_education_from_resume(s3))
print("CGPA:", extract_gpa_from_resume(s3))
print("Top roles that match candidate skills:", calculate_keyword_match(s3, jobRoles))

Name: Email Id
Contact Number: 9653392978
Email: zenianouphal5@gmail.com
Education: ['B.Tech']
CGPA: 9.17
Top roles that match candidate skills: ['Data Analyst', 'Business Analyst', 'Data Engineer']


In [24]:
res4=r"Resume_Mrunal11 - MRUNAL CHAVAN.pdf"
s4 = extract_text_from_pdf(res4)
print("Name:", extract_name_from_resume(s4))
print("Contact Number:", extract_contact_number_from_resume(s4))
print("Email:", extract_email_from_resume(s4))
print("Education:", extract_education_from_resume(s4))
print("CGPA:", extract_gpa_from_resume(s4))
print("Top roles that match candidate skills:", calculate_keyword_match(s4, jobRoles))

Name: Contact No
Contact Number: 8425808902
Email: mrunalchavan1973@gmail.com
Education: ['B.tech', 'B.Tech', 'B.tech', 'B.tech', 'B.tech']
CGPA: 8.9
Top roles that match candidate skills: ['Data Analyst', 'Machine Learning Engineer', 'Data Engineer']


In [25]:
res5=r"Resume - Rifat Perween.pdf"
s5 = extract_text_from_pdf(res5)
print("Name:", extract_name_from_resume(s5))
print("Contact Number:", extract_contact_number_from_resume(s5))
print("Email:", extract_email_from_resume(s5))
print("Education:", extract_education_from_resume(s5))
print("CGPA:", extract_gpa_from_resume(s5))
print("Top roles that match candidate skills:", calculate_keyword_match(s5, jobRoles))

Name: Sahil Raj
Contact Number: 8454800513
Email: raj17sahil@gmail.com
Education: ['Bachelors of Engineering in Computer Engineering']
CGPA: 8.73
Top roles that match candidate skills: ['Data Scientist', 'Blockchain Engineer', 'Data Analyst']


In [26]:
res6=r"Janhavi Resume - Janhavi Dixit.pdf"
s6 = extract_text_from_pdf(res6)
print("Name:", extract_name_from_resume(s6))
print("Contact Number:", extract_contact_number_from_resume(s6))
print("Email:", extract_email_from_resume(s6))
print("Education:", extract_education_from_resume(s6))
print("CGPA:", extract_gpa_from_resume(s6))
print("Top roles that match candidate skills:", calculate_keyword_match(s6, jobRoles))

Name: Janhavi Dixit
Contact Number: 9370680015
Email: janhavidixit249@gmail.com
Education: ['Bachelor of Technology']
CGPA: None
Top roles that match candidate skills: ['Data Analyst', 'Backend Engineer', 'Database Administrator']


In [27]:
res7=r"Resume-2 - 54_Nikita Shetty_IT3.pdf"
s7 = extract_text_from_pdf(res7)
print("Name:", extract_name_from_resume(s7))
print("Contact Number:", extract_contact_number_from_resume(s7))
print("Email:", extract_email_from_resume(s7))
print("Education:", extract_education_from_resume(s7))
print("CGPA:", extract_gpa_from_resume(s7))
print("Top roles that match candidate skills:", calculate_keyword_match(s7, jobRoles))

Name: Nikita Shetty
Contact Number: 9867332365
Email: shettynikit926@gmail.com
Education: ['Bachelor of Technology', 'B.Tech', 'Bachelor of Technology']
CGPA: 8.71
Top roles that match candidate skills: ['Backend Engineer', 'Frontend Engineer', 'Database Administrator']


In [28]:
res8=r"Sejal Ramchandrs Londhe  - Resume - 28_DS_sejal Londhe.pdf"
s8 = extract_text_from_pdf(res8)
print("Name:", extract_name_from_resume(s8))
print("Contact Number:", extract_contact_number_from_resume(s8))
print("Email:", extract_email_from_resume(s8))
print("Education:", extract_education_from_resume(s8))
print("CGPA:", extract_gpa_from_resume(s8))
print("Top roles that match candidate skills:", calculate_keyword_match(s8, jobRoles))

Name: Sejal Ramchandrs
Contact Number: 7715084590
Email: sejallondhe14@gmail.com
Education: []
CGPA: 7.66
Top roles that match candidate skills: ['Data Analyst', 'Backend Engineer', 'Business Analyst']


In [29]:
res9=r"Shriya Raina_Resume23 B1 - 48_Shriya Raina_IT.pdf"
s9 = extract_text_from_pdf(res9)
print("Name:", extract_name_from_resume(s9))
print("Contact Number:", extract_contact_number_from_resume(s9))
print("Email:", extract_email_from_resume(s9))
print("Education:", extract_education_from_resume(s9))
print("CGPA:", extract_gpa_from_resume(s9))
print("Top roles that match candidate skills:", calculate_keyword_match(s9, jobRoles))

Name: Usha Mittal
Contact Number: 9351031867
Email: shriyaraina556@gmail.com
Education: ['Bachelor of Technology', 'B.Tech']
CGPA: 7.35
Top roles that match candidate skills: ['Frontend Engineer', 'Data Analyst', 'Backend Engineer']


In [30]:
res10=r"Resume New - Purva Patil.pdf"
s10 = extract_text_from_pdf(res10)
print("Name:", extract_name_from_resume(s10))
print("Contact Number:", extract_contact_number_from_resume(s10))
print("Email:", extract_email_from_resume(s10))
print("Education:", extract_education_from_resume(s10))
print("CGPA:", extract_gpa_from_resume(s10))
print("Top roles that match candidate skills:", calculate_keyword_match(s10, jobRoles))

Name: Purva Atul
Contact Number: 9620538301
Email: purvap0078@gmail.com
Education: []
CGPA: None
Top roles that match candidate skills: ['Data Analyst', 'Data Scientist', 'Machine Learning Engineer']


In [31]:
res11=r"Nidhi Ravaria - 59_DS_Nidhi_ Ravariya.pdf"
s11 = extract_text_from_pdf(res11)
print("Name:", extract_name_from_resume(s11))
print("Contact Number:", extract_contact_number_from_resume(s11))
print("Email:", extract_email_from_resume(s11))
print("Education:", extract_education_from_resume(s11))
print("CGPA:", extract_gpa_from_resume(s11))
print("Top roles that match candidate skills:", calculate_keyword_match(s11, jobRoles))

Name: Nidhi Mavji
Contact Number: 9892700632
Email: nidhiravariya256@gmail.com
Education: ['B.Tech', 'Bachelor of Technology']
CGPA: 7.51
Top roles that match candidate skills: ['Data Analyst', 'Business Analyst', 'Machine Learning Engineer']


In [32]:
%%sql

INSERT INTO CANDIDATES(Name, Contact_Number, Email, Education, CGPA, Role1, Role2, Role3) 
VALUES ('Vansh Mehra', 9711982402, 'vansh_m@iitr.ac.in','B.Tech', 8.17, 'Data Analyst','Data  Engineer', 'Data Scientist'),
('Zenia Nouphal', 9653392978, 'zenianouphal5@gmail.com','B.Tech', 9.43, 'Data Analyst','Business Analyst', 'Data Engineer'),
('Mrunal Chavan', 8425808902, 'mrunalchavan1973@gmail.com','B.Tech', 8.9, 'Data Analyst','Machine Learning Engineer', 'Data Engineer'),
('Sahil Raj', 8454800513, 'raj17sahil@gmail.com','Bachelor of Engineering', 8.73, 'Data Scientist','Blockchain Engineer', 'Data Analyst'),
('Janhavi Dixit', 9370680015, 'janhavidixit249@gmail.com','Bachelor of Technology', 0.00, 'Data Analyst','Backend Engineer', 'Database Administrator'),
('Nikita Shetty', 9867332365, 'shettynikita926@gmail.com','Bachelor of Technology', 8.71, 'Backend Engineer','Frontend Engineer', 'Database Administrator'),
('Sejal Londhe', 7715084590, 'sejallondhe14@gmail.com','Null', 7.66, 'Data Analyst','Backend Engineer', 'Business Analyst'),
('Shriya Raina', 9351031867, 'shriyaraina556@gmail.com','Bachelor of Technology', 7.65, 'Frontend Engineer','Data Analyst', 'Backend Engineer'),
('Purva Patil', 9620538301, 'purvap0078@gmail.com','Null', 0.00, 'Data Analyst','Data Scientist', 'Machine Learning Engineer'),
('Nidhi Ravariya', 9892700632, 'nidhiravariya256@gmail.com','B.Tech', 7.51, 'Data Analyst','Business Analyst', 'Machine Learning Engineer');

 * sqlite:///SQLiteMagic.db
10 rows affected.


[]

#### 9. Query the table as per requirement.

In [34]:
%%sql 

SELECT * FROM CANDIDATES WHERE CGPA > 8.0;

 * sqlite:///SQLiteMagic.db
Done.


Name,Contact_Number,Email,Education,CGPA,Role1,Role2,Role3
Kirti Patel,8655204810,kirtiipatel4@gmail.com,Bachelor of Technology,9.43,Frontend Engineer,Data Analyst,Blockchain Engineer
Vansh Mehra,9711982402,vansh_m@iitr.ac.in,B.Tech,8.17,Data Analyst,Data Engineer,Data Scientist
Zenia Nouphal,9653392978,zenianouphal5@gmail.com,B.Tech,9.43,Data Analyst,Business Analyst,Data Engineer
Mrunal Chavan,8425808902,mrunalchavan1973@gmail.com,B.Tech,8.9,Data Analyst,Machine Learning Engineer,Data Engineer
Sahil Raj,8454800513,raj17sahil@gmail.com,Bachelor of Engineering,8.73,Data Scientist,Blockchain Engineer,Data Analyst
Nikita Shetty,9867332365,shettynikita926@gmail.com,Bachelor of Technology,8.71,Backend Engineer,Frontend Engineer,Database Administrator


In [35]:
%%sql 

SELECT * FROM CANDIDATES WHERE Role1 == 'Data Analyst';

 * sqlite:///SQLiteMagic.db
Done.


Name,Contact_Number,Email,Education,CGPA,Role1,Role2,Role3
Vansh Mehra,9711982402,vansh_m@iitr.ac.in,B.Tech,8.17,Data Analyst,Data Engineer,Data Scientist
Zenia Nouphal,9653392978,zenianouphal5@gmail.com,B.Tech,9.43,Data Analyst,Business Analyst,Data Engineer
Mrunal Chavan,8425808902,mrunalchavan1973@gmail.com,B.Tech,8.9,Data Analyst,Machine Learning Engineer,Data Engineer
Janhavi Dixit,9370680015,janhavidixit249@gmail.com,Bachelor of Technology,0.0,Data Analyst,Backend Engineer,Database Administrator
Sejal Londhe,7715084590,sejallondhe14@gmail.com,Null,7.66,Data Analyst,Backend Engineer,Business Analyst
Purva Patil,9620538301,purvap0078@gmail.com,Null,0.0,Data Analyst,Data Scientist,Machine Learning Engineer
Nidhi Ravariya,9892700632,nidhiravariya256@gmail.com,B.Tech,7.51,Data Analyst,Business Analyst,Machine Learning Engineer


In [36]:
%%sql 

SELECT * FROM CANDIDATES WHERE Education == 'B.Tech';

 * sqlite:///SQLiteMagic.db
Done.


Name,Contact_Number,Email,Education,CGPA,Role1,Role2,Role3
Vansh Mehra,9711982402,vansh_m@iitr.ac.in,B.Tech,8.17,Data Analyst,Data Engineer,Data Scientist
Zenia Nouphal,9653392978,zenianouphal5@gmail.com,B.Tech,9.43,Data Analyst,Business Analyst,Data Engineer
Mrunal Chavan,8425808902,mrunalchavan1973@gmail.com,B.Tech,8.9,Data Analyst,Machine Learning Engineer,Data Engineer
Nidhi Ravariya,9892700632,nidhiravariya256@gmail.com,B.Tech,7.51,Data Analyst,Business Analyst,Machine Learning Engineer


In [38]:
%%sql 

SELECT * FROM CANDIDATES WHERE Role1 == 'Machine Learning Engineer' OR Role2 == 'Machine Learning Engineer' OR Role3 == 'Machine Learning Engineer';

 * sqlite:///SQLiteMagic.db
Done.


Name,Contact_Number,Email,Education,CGPA,Role1,Role2,Role3
Mrunal Chavan,8425808902,mrunalchavan1973@gmail.com,B.Tech,8.9,Data Analyst,Machine Learning Engineer,Data Engineer
Purva Patil,9620538301,purvap0078@gmail.com,Null,0.0,Data Analyst,Data Scientist,Machine Learning Engineer
Nidhi Ravariya,9892700632,nidhiravariya256@gmail.com,B.Tech,7.51,Data Analyst,Business Analyst,Machine Learning Engineer
